In [1]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
import tensorflow as tf
import matplotlib.pyplot as plt
import time
from keras.preprocessing.image import ImageDataGenerator

/opt/conda/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:
metadata_path = './skin-cancer-mnist-ham10000/HAM10000_metadata.csv'
metadat = pd.read_csv(metadata_path)
num_samples = len(metadat)
metadat = metadat.sample(frac=1,random_state=12).reset_index(drop=True) #shuffling data

#metadat = metadat['image_id'].add('.jpg')
metadat.head()

,lesion_id,image_id,dx,dx_type,age,sex,localization
0,HAM_0003559,ISIC_0025810,bkl,consensus,75.0,male,face
1,HAM_0004240,ISIC_0032517,mel,histo,50.0,male,back
2,HAM_0002610,ISIC_0026876,vasc,consensus,0.0,female,abdomen
3,HAM_0003229,ISIC_0031140,bcc,histo,60.0,male,chest
4,HAM_0005963,ISIC_0028129,nv,follow_up,45.0,male,foot


In [7]:
# Use 20% test split (80% training + validation)
ntrain2 = int(len(metadat)*0.8)
ntrain1 = int(ntrain2*0.8)
x_test = metadat.iloc[ntrain2:,:]
x_train = metadat.iloc[:ntrain1,:]
x_val = metadat.iloc[ntrain1:ntrain2,:]

train_datagen = ImageDataGenerator(
        rescale=1./255,
        shear_range=0.2,
        zoom_range=0.2,
        horizontal_flip=True)

test_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_directory(
        './train',
        target_size=(299, 299),
        batch_size=32,
        seed=12)

validation_generator = test_datagen.flow_from_directory(
        './val',
        target_size=(299, 299),
        batch_size=32,
        seed=12)


Found 6409 images belonging to 7 classes.
Found 1602 images belonging to 7 classes.


In [9]:
from keras.applications.inception_v3 import InceptionV3
from keras.preprocessing import image
from keras.models import Model
from keras.layers import Dense, GlobalAveragePooling2D
from keras import backend as K

# create the base pre-trained model
base_model = InceptionV3(weights='imagenet', include_top=False)

# add a global spatial average pooling layer
x = base_model.output
x = GlobalAveragePooling2D()(x)
# let's add a fully-connected layer
x = Dense(1024, activation='relu')(x)
# and a logistic layer -- let's say we have 200 classes
predictions = Dense(7, activation='softmax')(x)

# this is the model we will train
model = Model(inputs=base_model.input, outputs=predictions)

# first: train only the top layers (which were randomly initialized)
# i.e. freeze all convolutional InceptionV3 layers
for layer in base_model.layers:
    layer.trainable = False

# compile the model (should be done *after* setting layers to non-trainable)
from keras.optimizers import Adam
model.compile(optimizer=Adam(lr=0.001, beta_1=0.9, beta_2=0.999, epsilon=0.1, decay=16/30, amsgrad=False), loss='categorical_crossentropy')

# train the model on the new data for a few epochs
model.fit_generator(train_generator,
        steps_per_epoch=len(train_generator)//32,
        epochs=50,
        validation_data=validation_generator,
        validation_steps=len(validation_generator)//32)

# at this point, the top layers are well trained and we can start fine-tuning
# convolutional layers from inception V3. We will freeze the bottom N layers
# and train the remaining top layers.

# let's visualize layer names and layer indices to see how many layers
# we should freeze:
for i, layer in enumerate(base_model.layers):
   print(i, layer.name)

# we chose to train the top 2 inception blocks, i.e. we will freeze
# the first 249 layers and unfreeze the rest:
for layer in model.layers[:249]:
   layer.trainable = False
for layer in model.layers[249:]:
   layer.trainable = True

# we need to recompile the model for these modifications to take effect
# we use SGD with a low learning rate
model.compile(optimizer=Adam(lr=0.001, beta_1=0.9, beta_2=0.999, epsilon=0.1, decay=16/30, amsgrad=False), loss='categorical_crossentropy')

# we train our model again (this time fine-tuning the top 2 inception blocks
# alongside the top Dense layers
model.fit_generator(train_generator,
        steps_per_epoch=len(train_generator)//32,
        epochs=50,
        validation_data=validation_generator,
        validation_steps=len(validation_generator)//32)

Epoch 1/50
6/6 [==============================] - 9s 2s/step - loss: 1.8935 - val_loss: 1.7719
Epoch 2/50
6/6 [==============================] - 4s 600ms/step - loss: 1.8232 - val_loss: 1.7251
Epoch 3/50
6/6 [==============================] - 7s 1s/step - loss: 1.7856 - val_loss: 1.6895
Epoch 4/50
6/6 [==============================] - 7s 1s/step - loss: 1.7040 - val_loss: 1.6605
Epoch 5/50
6/6 [==============================] - 7s 1s/step - loss: 1.6629 - val_loss: 1.6361
Epoch 6/50
6/6 [==============================] - 6s 996ms/step - loss: 1.5816 - val_loss: 1.6142
Epoch 7/50
6/6 [==============================] - 6s 1s/step - loss: 1.6408 - val_loss: 1.5949
Epoch 8/50
6/6 [==============================] - 7s 1s/step - loss: 1.6229 - val_loss: 1.5784
Epoch 9/50
6/6 [==============================] - 7s 1s/step - loss: 1.6224 - val_loss: 1.5641
Epoch 10/50
6/6 [==============================] - 6s 1s/step - loss: 1.5268 - val_loss: 1.5515
Epoch 11/50
6/6 [==========================

Epoch 1/50
6/6 [==============================] - 9s 2s/step - loss: 1.3271 - val_loss: 1.4272
Epoch 2/50
6/6 [==============================] - 2s 404ms/step - loss: 1.3369 - val_loss: 1.4167
Epoch 3/50
6/6 [==============================] - 6s 963ms/step - loss: 1.3399 - val_loss: 1.4090
Epoch 4/50
6/6 [==============================] - 5s 876ms/step - loss: 1.3004 - val_loss: 1.3981
Epoch 5/50
6/6 [==============================] - 6s 1s/step - loss: 1.3016 - val_loss: 1.3928
Epoch 6/50
6/6 [==============================] - 7s 1s/step - loss: 1.2272 - val_loss: 1.3890
Epoch 7/50
6/6 [==============================] - 7s 1s/step - loss: 1.3028 - val_loss: 1.3866
Epoch 8/50
6/6 [==============================] - 6s 1s/step - loss: 1.2213 - val_loss: 1.3844
Epoch 9/50
6/6 [==============================] - 5s 889ms/step - loss: 1.2244 - val_loss: 1.3809
Epoch 10/50
6/6 [==============================] - 6s 1s/step - loss: 1.2500 - val_loss: 1.3768
Epoch 11/50
6/6 [====================

In [10]:
model.save('./models/epoch50.h5')

In [ ]:
from keras.metrics import binary_accuracy
y_pred = model.predict_generator(validation_generator)
y_pred = np.argmax(y_pred,axis=0)
count = 0
correct = 0
for i in range(len(validation_generator)):
    if np.argmax(validation_generator[i][1]) == y_pred[i]:
        correct +=1
    count+=1
    print(str(acc))